<a href="https://colab.research.google.com/github/taaha3244/Finetune-LLMs/blob/main/Unsloth_llama3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install "unsloth[colab] @ git+https://github.com/unslothai/unsloth.git"

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-pghsc0ds/unsloth_10bb6fce3abc4ca69002a38d11d34472
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-pghsc0ds/unsloth_10bb6fce3abc4ca69002a38d11d34472
  Resolved https://github.com/unslothai/unsloth.git to commit 27fa021a7bb959a53667dd4e7cdb9598c207aa0d
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.8/211.8 MB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.4/102.4 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 45.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 67.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip install "git+https://github.com/huggingface/transformers.git"

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-m4v7wd0b
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-m4v7wd0b
  Resolved https://github.com/huggingface/transformers.git to commit 83238eeebc77e5a533803d72066f988f4707b9d5
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.42.0.dev0-py3-none-any.whl size=9140555 sha256=87bd5c3c6d5cfbf04d58e527ca39b5e0a3cc37b2318497b1ae639d385094dc8f
  Stored in directory: /tmp/pip-ephem-wheel-cache-cf2oi2p9/wheels/e7/9c/5b/e1a9c8007c343041e61cc484433d512ea9274272e3fcbe7c16
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.41.1
    Uninstalling transformers-4.41.1:
      Successfully uninstalled transformers-4.41.1


In [3]:
!pip install trl

In [25]:
!pip install huggingface_hub

In [5]:
from unsloth import FastLanguageModel
import torch
from trl import SFTTrainer
from transformers import TrainingArguments
from datasets import load_dataset

In [6]:
dataset = load_dataset("HuggingFaceH4/testing_self_instruct_small",split='train')

Generating train split:   0%|          | 0/100 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/100 [00:00<?, ? examples/s]

In [7]:
dataset

Dataset({
    features: ['prompt', 'completion'],
    num_rows: 100
})

In [8]:
max_seq_length=2048

In [11]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype=None,
    load_in_4bit=True
)

config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Llama patching release 2024.5
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.1.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.22.post7. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/172 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/464 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [12]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = True,
    random_state = 3407,
    max_seq_length = max_seq_length
)

Unsloth 2024.5 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [13]:

trainer = SFTTrainer(
    model = model,
    train_dataset = dataset,
    max_seq_length = max_seq_length,
    tokenizer = tokenizer,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 10,
        max_steps = 60,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        output_dir = "unsloth-test",
        optim = "adamw_8bit",
        seed = 3407,
    ),
)
trainer.train()

No chat template is set for this tokenizer, falling back to a default class-level template. This is very error-prone, because models are often trained with templates different from the class default! Default chat templates are a legacy feature and will be removed in Transformers v4.43, at which point any code depending on them will stop working. We recommend setting a valid chat template before then to ensure that this model continues working without issues.


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 100 | Num Epochs = 5
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,2.567000
2,2.666200
3,2.773100
4,2.125100
5,2.810500
6,2.436000
7,2.379200
8,2.527200
9,2.124900
10,2.256900


TrainOutput(global_step=60, training_loss=1.3847690929969152, metrics={'train_runtime': 287.4456, 'train_samples_per_second': 1.67, 'train_steps_per_second': 0.209, 'total_flos': 1911772528803840.0, 'train_loss': 1.3847690929969152, 'epoch': 4.8})

In [19]:

inputs = tokenizer(
    [
      "What is Mars?."
    ],
    return_tensors="pt",
).to("cuda")

In [20]:
inputs

{'input_ids': tensor([[128000,   3923,    374,  21725,   4710]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1]], device='cuda:0')}

In [21]:
outputs = model.generate(**inputs, max_new_tokens=128, use_cache=True)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [22]:
outputs

tensor([[128000,   3923,    374,  21725,   4710,  21725,    374,    279,  11999,
          11841,    505,    279,   8219,     13,  21725,    374,   1101,   2663,
            279,   3816,  29935,     13,  21725,    374,    279,   2132,  25655,
          11841,    304,    279,  13238,   1887,     13,  21725,    706,   1403,
          81260,     11,   2405,    677,    437,    323,   1611,  25075,     13,
          21725,    374,    922,   4376,    279,   1404,    315,   9420,    627,
          29986,    555,   3842,  37863,  30153,    927,    220,     19,   1667,
           4227,    198,  56339,    389,   7057,     25,    330,   3923,    374,
          21725,   4710,  21725,    374,    279,  11999,  11841,    505,    279,
           8219,     13,  21725,    374,   1101,   2663,    279,   3816,  29935,
             13,  21725,    374,    279,   2132,  25655,  11841,    304,    279,
          13238,   1887,     13,  21725,    706,   1403,  81260,     11,   2405,
            677,    437,    

In [23]:
tokenizer.batch_decode(outputs)

['<|begin_of_text|>What is Mars?. Mars is the fourth planet from the Sun. Mars is also called the Red Planet. Mars is the second smallest planet in the solar system. Mars has two moons, Phobos and Deimos. Mars is about half the size of Earth.\nPublished by John Edwards Modified over 4 years ago\nPresentation on theme: "What is Mars?. Mars is the fourth planet from the Sun. Mars is also called the Red Planet. Mars is the second smallest planet in the solar system. Mars has two moons, Phobos and Deimos. Mars is about half the size of Earth."— Presentation transcript:\n2 Mars is the fourth planet']

In [24]:
model.save_pretrained("lora_model")

In [26]:
from huggingface_hub import notebook_login

In [33]:

!huggingface-cli login



    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: write)

In [36]:
model.push_to_hub("taaha3244/unsloth-test",use_temp_dir=False)

README.md:   0%|          | 0.00/576 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

Saved model to https://huggingface.co/taaha3244/unsloth-test
